## Dependencies

---

In [1]:
import yaml
import re
from datetime import datetime
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException


Setting up web driver.

In [3]:
EdgeOptions = Options()
EdgeOptions.add_argument("--headless")
edge = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=EdgeOptions)

[WDM] - Downloading: 100%|██████████| 7.73M/7.73M [00:01<00:00, 4.97MB/s]


## Extract

---

### Functions:

In [4]:
def resources(name: str) -> dict[str:str]:
    """
    Read yaml file to get xpath resources
    :param name: name of the resource to extract
    :return: the xpath
    """
    with open("xpath.yml", "r") as f:
        content: dict[str:str] = yaml.safe_load(f)
        return content[name]

In [5]:
def close_prompt(driver: object) -> None:
    """
    Close prompt when is required
    :param driver: browser instance 
    """
    button: str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [6]:
def click_element(element: object, path: str) -> None:
    """
    Click on element in the current browser page
    :param element: browser instance
    :param path: xpath of element to click
    """
    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [7]:
def extract_feature_in(element: object, path: str) -> str:
    """
    Extract ad feature where current browser page is the ad page
    :param element: browser instance
    :param path: xpath of feature to extract
    :return: feature extracted
    """
    feature: str = element.find_element(by=By.XPATH, value=path)
    return feature

In [8]:
def extract_feature_out(element: object, path: str) -> list[str]:
    """
    Extract ad feature where current browser page is the listing page (not in the ad)
    :param element: browser instance
    :param path: xpath of feature to extract
    :return: list of features
    """
    feature = []

    try:
        feature: str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")

In [9]:
def persistence(dataframe: pd.DataFrame) -> None:
    """
    Save dataframe to csv file
    :param dataframe: dataframe to save
    """
    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [10]:
def get_len(features: list[list[str]]) -> bool:
    """
    Check if all features has the same length to create new df
    :param features: list of features extracted with scraping
    :return: a list with all lengths where first place is the len of the first element and the mostly accepted.
    """
    base_len = len(features[0])
    lengths = []
    lengths.append(base_len)
    for feature in features:
        if len(feature) == base_len:
            continue
        else:
            lengths.append(len(feature))
    return lengths

In [11]:
def check_len(lengths: list[int]) -> bool:
    """
    Check if all elements in list has the same value.
    :param lengths: list of lengths
    :return: a boolean
    """
    for length in lengths[1:]:
        if length != lengths[0]:
            return False
        return True

Variables pool.

In [12]:
descriptions: list[str] = []
prices: list[int] = []
models: list[str] = []
years: list[int] = []
kilometers: list[int] = []
engines: list[str] = []

# seconds
DELAY: int = 1
SEDAN = resources("cars_sedan")

Get web page and close prompt (when is required).

In [13]:
edge.get(SEDAN["url"])
# close_prompt(edge)

Extract car features.

In [14]:
# Get this list of ads and click on the first one
elements = edge.find_elements(by=By.XPATH, value=SEDAN["ad"])
elements[0].click()

try:
    for element in range(len(elements)):
        # Wait until the element is present to avoid missing data
        dynamic_delay = WebDriverWait(edge, DELAY).until(
            EC.presence_of_element_located((By.XPATH, SEDAN["description"])))

        brand_str = extract_feature_in(edge, SEDAN["brand"]).text
        model_str = extract_feature_in(edge, SEDAN["model"]).text
        models.append(brand_str + " " + model_str)

        descriptions.append(extract_feature_in(edge, SEDAN["description"]).text)
        prices.append(extract_feature_in(edge, SEDAN["price"]).text)
        kilometers.append(extract_feature_in(edge, SEDAN["kilometers"]).text)
        engines.append(extract_feature_in(edge, SEDAN["engine"]).text)
        years.append(extract_feature_in(edge, SEDAN["year"]).text)

        click_element(edge, SEDAN["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, SEDAN["exit"])

## Export

---

### Functions:

In [15]:
def parse_price(dataframe: pd.DataFrame, feature: str, pattern: str) -> None:
    """
    Parse financial features like prices. Works with $#,### (not dot)
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param pattern: feature pattern to transform
    """
    dataframe[feature] = dataframe[feature] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",", "")))


In [16]:
def remove_chars(dataframe: pd.DataFrame, feature: str, wildcards: str) -> pd.Series:
    """
    Remove escaped characters from features like indented large extracts of text
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param wildcards: escaped characters to remove
    :return: Serie without escaped characters
    """
    return (dataframe
    .apply(
        lambda row: row[feature]
        , axis=1
    )
    .apply(
        lambda extracted_f: re.sub(wildcards, r' ', extracted_f)
    )
    )


In [17]:
PRICE_RG: str = r"^\$(?P<prices>\d+,?\d+)?"
KM_REG: str = r"(?P<km>^\d+)"
CHARACTER_RG: str = '(\n|\r)+'

Check that all features has the same length.

In [18]:
lengths = [len(engines), len(models), len(years), len(kilometers), len(prices), len(descriptions)]
lengths

[43, 43, 43, 43, 43, 43]

Create dataframe to transform data extracted

In [19]:
if check_len(lengths):
    df_temp = pd.DataFrame({
        "model": models,
        "year": years,
        "kilometers": kilometers,
        "engine": engines,
        "price": prices,
        "description": descriptions
    })

    # Parse features. String to number and raw string with characters to structured string.
    parse_price(df_temp, "price", PRICE_RG)
    df_temp["kilometers"] = df_temp["kilometers"].str.extract(KM_REG).applymap(int)
    df_temp["year"] = df_temp["year"].map(int)
    df_temp["description"] = remove_chars(df_temp, "description", CHARACTER_RG)

    persistence(df_temp)


In [20]:
df_temp

,model,year,kilometers,engine,price,description
0,Volkswagen Jetta GLI,2014,78000,Gasolina,9000,Garantía: Como es visto no hay garantía Financ...
1,Nissan Sentra,2019,0,Gasolina,9600,Financiamiento: no Precio negociable: si Funci...
2,Hyundai Elantra,2018,78000,Gasolina,9900,"Precio negociable: si Color : AZUL $9,900.00 p..."
3,Ford Focus,2007,200000,Gasolina,3000,Garantía: Como es visto no hay garantía Color ...
4,Hyundai Accent,2017,114000,Gasolina,8000,Precio negociable: si DESCRIPCIÓN SEGURIDAD Ve...
5,Ford Fiesta,2015,80000,Gasolina,5700,Precio negociable: si DESCRIPCIÓN ARTÍCULOS CA...
6,Hyundai Veloster,2013,66000,Gasolina,7100,Precio negociable: si Color : Negro DESCRIPCIÓ...
7,Ford Focus,2013,141000,Gasolina,5000,Precio negociable: si Color : Negro DESCRIPCIÓ...
8,BMW 3 Series,2015,114000,Gasolina,15500,Color : Orion gray DESCRIPCIÓN CALIFICACIONES ...
9,Kia E,2000,0,Gasolina,2500,Financiamiento: si Precio negociable: si Color...
